In [1]:
import pandas as pd
import numpy as np
from openai import OpenAI
import pandas as pd
import numpy as np
import tiktoken
encoding = tiktoken.encoding_for_model("gpt-4")
from tqdm import tqdm
import pickle
from transformers import AutoModel, AutoTokenizer
from torch.nn.functional import cosine_similarity
import torch.nn.functional as F
import torch
import csv
import math
import pandas as pd
import re
import sys, getopt
import jsonlines
import random
import numpy as np
from tqdm import tqdm
import pandas as pd
import re
import numpy as np
import os
import time
from openai import OpenAI
import pandas as pd
import jsonlines
import numpy as np
from tqdm import tqdm
import json
import tiktoken
import ollama
encoding = tiktoken.encoding_for_model("gpt-4")
import os
import sys
import logging
from tqdm import tqdm
import csv 

/u1/hfaheem/.conda/envs/.env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Model, Prompt and DIR Initialisation

In [ ]:
# CHAGNE PROMPT HEREEEEEEE
with open('../prompts/prompt1-10.txt', 'r') as file:
    sys_prompt = file.read()

modelfile=f'''
FROM llama3.3
system """
{sys_prompt.strip()}
"""
'''
model = 'myexample1'
num_ctx = 25000 # 15000 original,  - works with 17000 with old GPUs 
ollama.create(model='myexample1', modelfile=modelfile)


input_folder = "./input"
csv_dir = input_folder + "/Execution-Paths/"
access_control_file = "/u1/hfaheem/DLAndroidArtifact/my-paths-code/input/Entrypoints.csv"
output_folder_preprocess = "./output/week1/output2"

In [ ]:
def process_csv_files(csv_directory):
    """
    Process all CSV files in the specified directory and structure the data into a DataFrame.
    
    Args:
        csv_directory (str): Path to directory containing CSV files
        
    Returns:
        pd.DataFrame: Structured data with service_name, class, method, and depths
    """
    data_dict = {}
    
    # Process each CSV file
    for filename in os.listdir(csv_directory):
        if filename.endswith('.csv'):
            service_name = os.path.splitext(filename)[0]
            filepath = os.path.join(csv_directory, filename)
            
            with open(filepath, 'r', newline='', encoding='utf-8') as csvfile:
                reader = csv.DictReader(csvfile)
                for row in reader:
                    try:
                        class_name = row['Class'].strip('"')
                        method = row['Method'].strip('"')
                        depth = row['Depth'].strip('"')
                        trace = row['Trace Instruction(s) ...'].strip('"')
                        java_code = row['Java Code Representation'].strip('"')
                        access_control_level = row['Access Control Level'].strip('"')
                        # if access_control_level == 'NONE':
                        #     print(f"found it: {filename}")
                        # Create unique key for grouping
                        key = (service_name, class_name, method, access_control_level)
                        
                        # Create depth entry
                        depth_entry = {
                            'depth': int(depth),
                            'trace': trace,
                            'java_code': java_code
                        }
                        
                        # Add to data dictionary
                        if key not in data_dict:
                            data_dict[key] = []
                        data_dict[key].append(depth_entry)
                        
                    except KeyError as e:
                        print(f"Missing column in {filename}: {e}")
                        continue
    
    # Convert to DataFrame
    rows = []
    for key in data_dict:
        service, cls, method, access_control_level = key
        depths = sorted(data_dict[key], key=lambda x: x['depth'])  # Sort by depth
        rows.append({
            'service_name': service,
            'class': cls,
            'method': method,
            'depths': depths,
            'access control level': access_control_level
        })
    
    return pd.DataFrame(rows)

def get_java_code(row):
# get the maximum depth
    max_depth = max([x['depth'] for x in row["depths"]])
    # get all the java code for the max_depth
    code_string = ""
    counter = 1
    for code in row["depths"]:
        if code['depth'] == max_depth:
            code_string += f"This is path {counter} for the API with depth {max_depth}:\n"
            code_string += code['java_code']
            counter += 1

    return code_string

def run_first_prompt_Ollama(method_code,run):
    """ runs the first prompt - extract sinks from the traces
    """
    
    global sys_prompt

    # user_prompt = get_method_traces_from_file(file_path, interface, method)
    
    user_prompt = method_code
    if(not run):
        return {
            "system_message": sys_prompt,
            "user_message": user_prompt,
            "response": """
                    After analyzing the provided Java code for the `addInstrumentationResults` API, I have identified one potential sink. Here is the JSON output:

```
{
  "Sinks": [
    [
      "int var5 = UserHandle.getCallingUserId();",
      "ProcessRecord var16 = getRecordForAppLocked(param2);",
      "IApplicationThread var26 = var25.thread;",
      "IApplicationThread var27 = var25.thread;"
    ]
  ]
}
```

Ranking: This sink is ranked as the most sensitive.
    
            """
        }
        
        
    
    response = ollama.chat(model=model, messages=[
    {
        'role': 'user',
        'content': user_prompt,
    },
    ]
    ,
     options={
        'num_ctx': num_ctx,
        # 'temperature': 0.3 # Todo : CHECK TEMPERATURE HERERERERERERERER
    }
    
    )
    
    # logging.info(f"Response for {method} = {response['message']['content']}")

    
    return {
        "system_message": sys_prompt,
        "user_message": user_prompt,
        "response": response['message']['content']
    }

def extract_json_from_string(input_string):
    """
    Extracts JSON from the given string.

    Args:
        input_string (str): The string containing embedded JSON.

    Returns:
        dict: The extracted JSON as a Python dictionary.
    """
    try:
        # Use a regex pattern to extract the JSON part
        json_pattern = r"```(?:json)?\n(.*?)\n```"
        match = re.search(json_pattern, input_string, re.DOTALL)
        if match:
            json_string = match.group(1)
            return json.loads(json_string)
        else:
            raise ValueError("No JSON found in the provided string.")
    except json.JSONDecodeError as e:
        raise ValueError(f"Error decoding JSON: {e}")

def process_dataframe(df):
    
    global output_folder_preprocess  # Use global variables

    df['json_answer'] = None  # Initialize a new column for JSON answers
    
    for index, row in tqdm(df.iterrows(), total=df.shape[0], desc="Processing rows"):
        method_name = row['method'].split("(")[0]
        service_name = row['service_name']
        code_string = get_java_code(row)

        res = run_first_prompt_Ollama(code_string, True)
        json_answer = None

        try:
            json_answer = extract_json_from_string(res["response"])
        except Exception as e:
            print(f"Error extracting JSON from response: {e}")
            print(f"Method: {method_name}, Service: {service_name}")

        df.at[index, 'json_answer'] = json_answer  # Store JSON answer in DataFrame

        # Define folder path and create directories
        folder_path = os.path.join(output_folder_preprocess, service_name, method_name)
        os.makedirs(folder_path, exist_ok=True)

        # Write response details to files
        with open(os.path.join(folder_path, 'system_message.txt'), 'w') as f:
            f.write(res["system_message"])
        with open(os.path.join(folder_path, 'user_message.txt'), 'w') as f:
            f.write(res["user_message"])
        with open(os.path.join(folder_path, 'response.txt'), 'w') as f:
            f.write(res["response"])

    # Save the updated DataFrame to a Parquet file
    df_output_file = os.path.join(output_folder_preprocess, "android_services_methods.parquet")
    df.to_parquet(df_output_file)
    print(f"DataFrame serialized and saved to {df_output_file}")

    return df  # Return the updated DataFrame

In [4]:
df = process_csv_files(csv_dir)
df["service_name"].value_counts()
df[df["service_name"] == "Lcom.android.server.pm.UserManagerService"]["access control level"].value_counts()
df_ums = df[df["service_name"] == "Lcom.android.server.pm.UserManagerService"]

In [ ]:
process_dataframe(df_ums)

PostProcess

In [34]:
file_path = "/u1/hfaheem/DLAndroidArtifact/my-paths-code/output/week1/output1/android_services_methods.parquet"
CSV_FILE = './similaritiescheck'

In [ ]:

checkpoint="Salesforce/codet5p-110m-embedding"
tokenizer = AutoTokenizer.from_pretrained(checkpoint, trust_remote_code=True)
model = AutoModel.from_pretrained(checkpoint, trust_remote_code=True).to("cuda")
def get_code_embedding(code_snippet):
    """
    Generates embeddings for a given code snippet using a pre-trained model.

    Parameters:
    - code_snippet (str): The code for which embeddings are to be generated.
    - checkpoint (str): The model checkpoint to be used for embedding. Default is Salesforce/codet5p-110m-embedding.
    - device (str): Device to run the model on, either 'cuda' for GPU or 'cpu' for CPU. Default is 'cuda'.

    Returns:
    - torch.Tensor: Embedding tensor for the input code.
    """
    inputs = tokenizer.encode(code_snippet, return_tensors="pt").to("cuda")
    with torch.no_grad():
        embedding = model(inputs)[0]
    
    return embedding.cpu()

def process_json_answer(json_answer, n=2):
    global counter
    all = []
    try:
        arrays = json_answer['Sinks']
        for i, array in enumerate(arrays, 1):
            if i > n:  # Limit the number of joins to `n`
                break
            joined = '\n'.join(array)
            all.append(joined)
    except:    
        return []
    counter += 1
    return all

def calculate_embeddings(df):
    df["embeddings"] = None
    for index, row in tqdm(df.iterrows(), total=len(df), desc="Processing Rows"):
        code_snippets = row["sink_code"]
        embeddings = []
        for each in code_snippets:
            code_embedding = get_code_embedding(each)
            embeddings.append(code_embedding)
        df.at[index, "embeddings"] = embeddings



Calculate Embeddings

In [24]:
counter = 0
df = pd.read_parquet(file_path)
df['sink_code'] = df['json_answer'].apply(process_json_answer)
print(f"total rows = {len(df)}")
print(f"row with valid json  = {counter}")

calculate_embeddings(df)

total rows = 85
row with valid json  = 82


Processing Rows: 100%|██████████| 85/85 [00:01<00:00, 69.98it/s]


In [25]:
df.columns

Index(['service_name', 'class', 'method', 'depths', 'access control level',
       'json_answer', 'sink_code', 'embeddings'],
      dtype='object')

In [32]:
def compute_most_similar(df):
    """
    Computes the most similar method for each method in the DataFrame based on cosine similarity of embeddings.

    Parameters:
    - df (pd.DataFrame): DataFrame containing 'method', 'embeddings', and 'sink_code'.

    Returns:
    - dict: A dictionary where each method maps to its most similar method, similarity score, and matching code pair.
    """
    similarities = {}

    for index1, row1 in tqdm(df.iterrows(), total=len(df), desc="Computing Similarities"):
        ep1_id = row1["method"]  
        ep1_embeddings = row1["embeddings"]  # List of embeddings
        ep1_sink_code = row1["sink_code"]  # Corresponding code snippets
        
        closest_ep = None
        max_similarity = -1
        similar_sink_pair = None
        
        for index2, row2 in df.iterrows():
            if index1 == index2:
                continue  
            
            ep2_id = row2["method"]
            ep2_embeddings = row2["embeddings"]
            ep2_sink_code = row2["sink_code"]
            
            # Compute pairwise similarities
            for i, emb1 in enumerate(ep1_embeddings):
                for j, emb2 in enumerate(ep2_embeddings):
                    sim = torch.nn.functional.cosine_similarity(torch.tensor(emb1), torch.tensor(emb2), dim=0).item()
                    if sim > max_similarity:
                        max_similarity = sim
                        closest_ep = ep2_id
                        similar_sink_pair = (ep1_sink_code[i], ep2_sink_code[j])  
        
        similarities[ep1_id] = {
            "closest_ep": closest_ep,
            "max_similarity": max_similarity,
            "similar_sink_pair": similar_sink_pair
        }
    
    return similarities

# to print
# for ep, details in similarities.items():
#     if details['similar_sink_pair'] is None or details['closest_ep'] is None:
#         print(f"EP {ep} has no similar EPs.")
#     else:
#         print(f"EP {ep} is most similar to EP {details['closest_ep']} with similarity {details['max_similarity']:.4f}")
#         print(f"Similar sink codes:\nEP {ep}: \n{details['similar_sink_pair'][0]}\nEP \n{details['closest_ep']}: \n{details['similar_sink_pair'][1]}")
#     print("\n\n")



def compute_80_top2(df):
    similarities = {}

    for index1, row1 in tqdm(df.iterrows(), total=len(df), desc="Computing Similarities"):
        ep1_id = row1["method"] # name of the EP1
        ep1_embeddings = row1["embeddings"] # list of embeddings for the EP1
        ep1_sink_code = row1["sink_code"] # list of code snippets for the EP1
        
        similar_sink_pairs = []  # List to store all similar sink code pairs with similarity > 0.8
        
        for index2, row2 in df.iterrows():
            if index1 == index2:
                continue  
            
            ep2_id = row2["method"] # name of the EP2
            ep2_embeddings = row2["embeddings"] # list of embeddings for the EP2
            ep2_sink_code = row2["sink_code"] # list of code snippets for the EP2  
            
            # Compute similarities
            for i, emb1 in enumerate(ep1_embeddings):
                for j, emb2 in enumerate(ep2_embeddings):
                    sim = torch.nn.functional.cosine_similarity(torch.tensor(emb1), torch.tensor(emb2), dim=0).item()
                    if sim > 0.8:  # Only consider pairs with similarity > 0.8
                        similar_sink_pairs.append({
                            "similarity": sim,
                            "ep1_code": ep1_sink_code[i],
                            "ep2_id": ep2_id,
                            "ep2_code": ep2_sink_code[j]
                        })
        
        # Store all similar pairs for the current EP
        similarities[ep1_id] = similar_sink_pairs
    return similarities
        

def write_csvs(similarities):
    global CSV_FILE  # Use the global CSV_FILE variable
    # Write the data to a CSV file
    with open(CSV_FILE+ "_allcode" + ".csv", mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)

        # Write header
        writer.writerow(["EP1_ID", "EP2_ID", "EP1_Code", "EP2_Code", "Similarity"])

        # Iterate through the data and write each entry
        for ep, similar_pairs in similarities.items():
            if similar_pairs:
                for pair in similar_pairs:
                    writer.writerow([
                        ep,
                        pair['ep2_id'],
                        pair['ep1_code'].replace("\n", " "),  # Replace newlines for better CSV readability
                        pair['ep2_code'].replace("\n", " "),  # Replace newlines for better CSV readability
                        pair['similarity']
                    ])
            else:
                writer.writerow([ep, "No similar EPs with similarity > 0.8", "", "", ""])

    # Writing the data to a CSV file
    with open(CSV_FILE + "_score" + ".csv", mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        # Write the header
        writer.writerow(["EP", "EP2_ID", "Max_Similarity"])
        
        for ep, similar_pairs in similarities.items():
            if not similar_pairs:
                writer.writerow([ep, "No similar EPs", ""])
            else:
                max_similarity_per_ep2 = {}
                for pair in similar_pairs:
                    ep2_id = pair['ep2_id']
                    similarity = pair['similarity']
                    if ep2_id not in max_similarity_per_ep2:
                        max_similarity_per_ep2[ep2_id] = similarity
                    else:
                        max_similarity_per_ep2[ep2_id] = max(max_similarity_per_ep2[ep2_id], similarity)
                
                # Sorting the EP2 IDs by similarity in descending order
                sorted_ep2_ids = sorted(max_similarity_per_ep2.items(), key=lambda x: x[1], reverse=True)
                
                # Writing sorted EP2 IDs and their max similarity
                for ep2_id, max_similarity in sorted_ep2_ids:
                    writer.writerow([ep, ep2_id, f"{max_similarity:.4f}"])



    with open(CSV_FILE + "_top2code" + ".csv", mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)

        # Write header
        writer.writerow(["EP1_ID", "EP2_ID", "EP1_Code", "EP2_Code", "Similarity"])

        # Iterate through the data and write only the top 2 most similar pairs
        for ep, similar_pairs in similarities.items():
            
            if similar_pairs:
                # Sort the pairs by similarity in descending order
                top_pairs = sorted(similar_pairs, key=lambda x: x["similarity"], reverse=True)[:2]
                for pair in top_pairs:
                    writer.writerow([
                        ep,
                        pair['ep2_id'],
                        pair['ep1_code'].replace("\n", " "),  # Replace newlines for better CSV readability
                        pair['ep2_code'].replace("\n", " "),  # Replace newlines for better CSV readability
                        pair['similarity']
                    ])
            else:
                writer.writerow([ep, "No similar EPs with similarity > 0.8", "", "", ""])

    print(f"Data has been written to {CSV_FILE}")

In [36]:
similarities = compute_80_top2(df)
write_csvs(similarities)


Computing Similarities:   0%|          | 0/85 [00:00<?, ?it/s]/tmp/ipykernel_260751/740237442.py:79: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sim = torch.nn.functional.cosine_similarity(torch.tensor(emb1), torch.tensor(emb2), dim=0).item()
Computing Similarities: 100%|██████████| 85/85 [00:01<00:00, 69.90it/s]

Data has been written to ./similaritiescheck


In [37]:
similarities

{'getUserInfo(I)Landroid/content/pm/UserInfo;': [],
 'getUserBadgeResId(I)I': [{'similarity': 0.8754937052726746,
   'ep1_code': 'p0.checkManageOrInteractPermissionIfCallerInOtherProfileGroup(p1, "getUserBadgeResId");\nboolean v2 = p0.isSameProfileGroupNoChecks(v1, p1);\nint v4 = Binder.getCallingUid();\nboolean v5 = UserManagerService.hasPermissionGranted("android.permission.INTERACT_ACROSS_USERS", v4)',
   'ep2_id': 'hasBadge(I)Z',
   'ep2_code': 'p0.checkManageOrInteractPermissionIfCallerInOtherProfileGroup(p1, "hasBadge");'},
  {'similarity': 0.8949418067932129,
   'ep1_code': 'p0.checkManageOrInteractPermissionIfCallerInOtherProfileGroup(p1, "getUserBadgeResId");\nboolean v2 = p0.isSameProfileGroupNoChecks(v1, p1);\nint v4 = Binder.getCallingUid();\nboolean v5 = UserManagerService.hasPermissionGranted("android.permission.INTERACT_ACROSS_USERS", v4)',
   'ep2_id': 'isUserRunning(I)Z',
   'ep2_code': 'p0.checkManageOrInteractPermissionIfCallerInOtherProfileGroup(p1, "isUserRunning")

In [39]:
df.columns

Index(['service_name', 'class', 'method', 'depths', 'access control level',
       'json_answer', 'sink_code', 'embeddings'],
      dtype='object')

In [61]:
def get_top_similar_methods(similarities, top_n=2, threshold=0.8):
    filtered = [entry for entry in similarities if entry['similarity'] > threshold]
    sorted_results = sorted(filtered, key=lambda x: x['similarity'], reverse=True)
    top_results = sorted_results[:top_n]
    extracted_results = [
        {
            'ep2_code': entry['ep2_code'],
            'ep2_id': entry['ep2_id'],
            'ep1_code': entry['ep1_code'],
            'similarity': entry['similarity']
        }
        for entry in top_results
    ]
    
    return extracted_results

def create_prompt2_string(top_similar, df, original_method):
    if not top_similar:
        return "No similar APIs found"  # Return an empty string if there are no similar methods
    
    prompt = f"The original method {original_method} has the following similar APIs:\n\n"

    for entry in top_similar:
        ep2_id = entry["ep2_id"]
        ep2_code = entry["ep2_code"]

        # Find access control level for ep2_id
        access_control = df[df["method"] == ep2_id]["access control level"].values
        access_control_str = access_control[0] if len(access_control) > 0 else "Unknown"

        prompt += f"- API Name: {ep2_id} with"
        prompt += f" Similarity Score: {entry.get('similarity', 'N/A')}\n"
        prompt += f"  - Access Control Level: {access_control_str} and the following code:\n"
        prompt += f"{ep2_code}\n\n"

    # Add final instructions for assigning access control
    prompt += (
        "Given this information, your task is to assign an access control to the "
        f"{original_method} API. After the explanation, give the final access control "
        "level for the API in JSON format as follows:\n"
        "{ \"access_control_level\": \"...\" }.\n\n"
        "You have 4 choices: NONE, NORMAL, DANGEROUS, SYS_OR_SIG. To make an informed decision, "
        "carefully review other APIs (ground truth) that I have provided above that interact "
        "with the same sinks, their assigned access control levels, and the semantics of those APIs. "
    )

    return prompt
    

In [ ]:

counter = 0
for index, row in tqdm(df.iterrows(), total=df.shape[0], desc="Processing rows"):
    full_method_name = row['method']
    method_name = row['method'].split("(")[0]
    service_name = row['service_name']
    
    # print(method_name, service_name)
    # print(similarities[full_method_name])
    # print(get_top_similar_methods(similarities[full_method_name]))
    
    top_similar = get_top_similar_methods(similarities[full_method_name])
    
    Access_Control = ""
    prompt = ""
    res = []
    if top_similar == "No similar APIs found":
        Access_Control = "invalid"
        res = {"user_message":"invalid", "response":"invalid"}
    
    else:
        prompt= create_prompt2_string(top_similar, df, method_name)
        
        # todo: implement run prompt
        # run prompt
        # get json
    
    folder_path = f'{output_folder_preprocess}/{service_name}/{method_name}'
    os.makedirs(folder_path, exist_ok=True)
    with open(f'{folder_path}/user_message2.txt', 'w') as f:
        f.write(res["user_message"])
    with open(f'{folder_path}/response2.txt', 'w') as f:
        f.write(res["response"])
    
    
    counter += 1
    if counter > 1:
        break
    
    continue
    code_string = get_java_code(row)

    res = run_first_prompt_Ollama(code_string, True)
    json_answer = None

    try:
        json_answer = extract_json_from_string(res["response"])
    except Exception as e:
        print(f"Error extracting JSON from response: {e}")
        print(f"Method: {method_name}, Service: {service_name}")

    df.at[index, 'json_answer'] = json_answer  # Store JSON answer in DataFrame

    # Define folder path and create directories
    folder_path = os.path.join(output_folder_preprocess, service_name, method_name)
    os.makedirs(folder_path, exist_ok=True)

    # Write response details to files
    with open(os.path.join(folder_path, 'system_message.txt'), 'w') as f:
        f.write(res["system_message"])
    with open(os.path.join(folder_path, 'user_message.txt'), 'w') as f:
        f.write(res["user_message"])
    with open(os.path.join(folder_path, 'response.txt'), 'w') as f:
        f.write(res["response"])

# Save the updated DataFrame to a Parquet file
# df_output_file = os.path.join(output_folder_preprocess, "android_services_methods.parquet")
# df.to_parquet(df_output_file)
# print(f"DataFrame serialized and saved to {df_output_file}")


Processing rows:   1%|          | 1/85 [00:00<00:00, 758.74it/s]

Nothing found
The original method getUserBadgeResId has the following similar APIs:

- API Name: getUserBadgeLabelResId(I)I with Similarity Score: 0.9809373021125793
  - Access Control Level: SYS_OR_SIG and the following code:
p0.checkManageOrInteractPermissionIfCallerInOtherProfileGroup(p1, "getUserBadgeLabelResId");
boolean v2 = p0.isSameProfileGroupNoChecks(v1, p1);
boolean v3 = UserManagerService.hasManageUsersPermission();
int v4 = Binder.getCallingUid();
boolean v5 = UserManagerService.hasPermissionGranted("android.permission.INTERACT_ACROSS_USERS", v4);

- API Name: getUserBadgeColorResId(I)I with Similarity Score: 0.9789021015167236
  - Access Control Level: SYS_OR_SIG and the following code:
p0.checkManageOrInteractPermissionIfCallerInOtherProfileGroup(p1, "getUserBadgeColorResId");
boolean v2 = p0.isSameProfileGroupNoChecks(v1, p1);
boolean v3 = UserManagerService.hasManageUsersPermission();
int v4 = Binder.getCallingUid();
boolean v5 = UserManagerService.hasPermissionGranted